In [1]:
import os
import sys
import glob
from os import write

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast
import scipy.stats as st
from scipy.stats import pearsonr, zscore

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities

from iDrink.iDrinkUtilities import get_title_measure_name, get_unit, get_cad, get_setting_axis_name, get_measure_short_name

In [2]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_stat_cat = os.path.join(root_stat, '02_categorical')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots_cont = os.path.join(dir_stat_cont, '02_plots')
dir_plots_cat = os.path.join(dir_stat_cat, '02_plots')

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')
df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']  # Becaus bad calibration reprojection error

murphy_measures = ['PeakVelocity_mms', 'elbowVelocity', 'tTopeakV_s', 'tToFirstpeakV_s',
       'tTopeakV_rel', 'tToFirstpeakV_rel', 'NumberMovementUnits',
       'InterjointCoordination', 'trunkDisplacementMM', 'trunkDisplacementDEG',
       'ShoulderFlexionReaching', 'ElbowExtension', 'shoulderAbduction',
       'shoulderFlexionDrinking']

ignore_id_p = ['P11', 'P15', 'P19']
idx_s_singlecam_full = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']
idx_s_singlecam = ['S017', 'S018']
idx_s_multicam = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010', 'S011', 'S012', 'S013', 'S014', 'S015', 'S016']
idx_s_multicam_reduced = ['S001', 'S002']
idx_s_reduced = idx_s_multicam_reduced + idx_s_singlecam
idx_s_full = idx_s_multicam + idx_s_singlecam

reduced_analysis = True # If True, only reduced set of subjects will be analyzed

# Bland Altman Tables
## Kinematic Measures

In [3]:
df_murphy = df_murphy[df_murphy['id_p'].isin(ignore_id_p) == False]
df_murphy.drop(columns=['valid', 'ReachingStart',
       'ForwardStart', 'DrinkingStart', 'BackStart', 'ReturningStart',
       'RestStart', 'TotalMovementTime'], inplace=True)

In [4]:
df_murphy.columns

Index(['identifier', 'id_s', 'id_p', 'id_t', 'side', 'condition',
       'PeakVelocity_mms', 'elbowVelocity', 'tTopeakV_s', 'tToFirstpeakV_s',
       'tTopeakV_rel', 'tToFirstpeakV_rel', 'NumberMovementUnits',
       'InterjointCoordination', 'trunkDisplacementMM', 'trunkDisplacementDEG',
       'ShoulderFlexionReaching', 'ElbowExtension', 'shoulderAbduction',
       'shoulderFlexionDrinking'],
      dtype='object')

In [5]:
df_altman = pd.DataFrame(columns=['id_s', 'id_p', 'measure', 'mean_err', 'lower_loa', 'upper_loa'])

In [6]:
cols_error = ['PeakVelocity_mms', 'elbowVelocity', 'tTopeakV_s',
              'tToFirstpeakV_s', 'tTopeakV_rel', 'tToFirstpeakV_rel', 'NumberMovementUnits',
              'InterjointCoordination', 'trunkDisplacementMM', 'trunkDisplacementDEG', 'ShoulderFlexionReaching',
              'ElbowExtension', 'shoulderAbduction', 'shoulderFlexionDrinking']
df_murphy_full = df_murphy.copy()
df_murphy = df_murphy[(np.abs(zscore(df_murphy[cols_error])) < 3).all(axis=1)].reindex()

In [7]:
df_murphy_omc_s = df_murphy[df_murphy['id_s'] == 'S15133']
df_murphy_mmc = df_murphy[df_murphy['id_s'] != 'S15133']

if reduced_analysis:
    df_murphy_mmc = df_murphy_mmc[df_murphy_mmc['id_s'].isin(idx_s_reduced)]
    df_murphy_mmc = df_murphy_mmc[~df_murphy_mmc['id_p'].isin(ignore_id_p)]
    df_murphy_omc_s = df_murphy_omc_s[~df_murphy_omc_s['id_p'].isin(ignore_id_p)]




idx_s = df_murphy_mmc['id_s'].unique()
idx_p = set(df_murphy_omc_s['id_p'].unique()).intersection(set(df_murphy_mmc['id_p'].unique()))

In [8]:
df_murphy_full.shape

(6418, 20)

In [9]:
df_murphy.shape

(0, 20)

In [10]:
df_murphy_mmc.shape

(0, 20)

In [11]:
make_plots = False

In [12]:
df_altman = pd.DataFrame(columns=['id_s', 'id_p', 'measure', 'mean_err', 'lower_loa', 'upper_loa'])
total = len(idx_s) * len(murphy_measures)

progbar = tqdm(total=total, desc='Processing')
for measure in murphy_measures:
    
    measure_name = get_title_measure_name(measure)
    unit = get_unit(measure)
    colors = px.colors.qualitative.Plotly
    
    for id_s in idx_s:
        
        progbar.set_description(f'Processing {id_s} \t {measure}')
        
        df_murphy_mmc_s = df_murphy_mmc[df_murphy_mmc['id_s'] == id_s]
        i = 0
        
        if make_plots:
            fig_val_val = go.Figure()
        max_val = 0
        
        df_omc_fit = None
        df_mmc_fit = None
        
        id_s_name = get_setting_axis_name(id_s)
        
        for id_p in idx_p:
            df_murphy_omc_p = df_murphy_omc_s[df_murphy_omc_s['id_p'] == id_p]
            df_murphy_mmc_p = df_murphy_mmc_s[df_murphy_mmc_s['id_p'] == id_p]
            
            idx_t = set(df_murphy_omc_p['id_t'].unique()).intersection(set(df_murphy_mmc_p['id_t'].unique()))
        
            df_murphy_omc_p = df_murphy_omc_p[df_murphy_omc_p['id_t'].isin(idx_t)].sort_values(by='id_t')
            df_murphy_mmc_p = df_murphy_mmc_p[df_murphy_mmc_p['id_t'].isin(idx_t)].sort_values(by='id_t')
            
            if df_omc_fit is None:
                df_omc_fit = df_murphy_omc_p
                df_mmc_fit = df_murphy_mmc_p
            else:
                df_omc_fit = pd.concat([df_omc_fit, df_murphy_omc_p], ignore_index=True)
                df_mmc_fit = pd.concat([df_mmc_fit, df_murphy_mmc_p], ignore_index=True)
            
            # get values and add to DataFrame
            means = np.nanmean([df_murphy_omc_p[measure].values, df_murphy_mmc_p[measure].values], axis=0)
            
            diff = df_murphy_mmc_p[measure].values - df_murphy_omc_p[measure].values
            mean_err = np.nanmean(diff)
            std = np.nanstd(diff, ddof=1)
            unit = get_unit(measure)
            
            # Confidence level
            confidence = 0.95
            loas = st.norm.interval(confidence, mean_err, std)
            lower_loa = loas[0]
            upper_loa = loas[1]
            
            std_diff = np.std(diff)
            sd = 1.96
            upper_loa = mean_err + sd * std_diff
            lower_loa = mean_err - sd * std_diff
            
            df_altman = pd.concat([df_altman, pd.DataFrame({'id_s': id_s, 'id_p': id_p, 'measure': measure, 'mean_err': mean_err, 'lower_loa': lower_loa, 'upper_loa': upper_loa}, index = [0])], ignore_index=True)
            
            # Make Value-Value Plot
            
            if make_plots:
                try:
                    new_max = max(np.nanmax(df_murphy_mmc_p[measure].values), np.nanmax(df_murphy_omc_p[measure].values))
                    if max_val < new_max:
                        max_val = new_max
                except:
                    pass
                
            
                fig_val_val.add_trace(go.Scatter(x=df_murphy_mmc_p[measure].values, y=df_murphy_omc_p[measure].values, mode='markers', name=f'{id_p}', marker=dict(color=colors[i], size=10)))
                fig_val_val.update_layout(title=f'{measure_name} for {id_s_name}', xaxis_title=f'MMC {measure_name} [{unit}]', yaxis_title=f'OMC {measure_name} [{unit}]', template='plotly')
                
            i+=1
        
        if make_plots:    
            fig_val_val.add_trace(go.Scatter(x=[0, max_val], y=[0, max_val], mode='lines', name='Line of Equality', line=dict(color='grey', width=2, dash='dash')))
            
            try:
                regress = st.linregress(x=df_mmc_fit[measure].values, y=df_omc_fit[measure].values)
                fig_val_val.add_trace(go.Scatter(x=df_mmc_fit[measure].values, y=regress.intercept + regress.slope * df_mmc_fit[measure].values, mode='lines', name='Regression Line', line=dict(color='red', width=2)))
                
                fig_val_val.update_layout(template='plotly', height = 1000, width = 1000)
            
                fig_val_val.add_annotation(
                    x=max_val,  # Set the annotation near the maximum x value
                    y=0,  # Set the annotation near the minimum y value
                    text=f"Slope: {regress.slope:.2f}",  # Display the slope with 2 decimal places
                    showarrow=False,  # No arrow needed
                    font=dict(size=12),  # Adjust font size
                    xanchor="right",  # Align to the right
                    yanchor="bottom"  # Align to the bottom
                    )
            except:
                pass
        
        # If nregress is None, do not write the image
        
        progbar.update(1)
progbar.close()

df_altman.dropna()

Processing: 0it [00:00, ?it/s]


,id_s,id_p,measure,mean_err,lower_loa,upper_loa


## Make Plot for Powerpoint

In [13]:
df_altman

,id_s,id_p,measure,mean_err,lower_loa,upper_loa


In [14]:
dir_out = r'C:\Users\johan\OneDrive\Dokumente\Studium\LLUI_Praktikum\09 Masterarbeit\02 Präsentationen\Final Presentation'

id_s_for_powerpoint = ['S002', 'S017']

df_altman_powerpoint = df_altman[df_altman['id_s'].isin(id_s_for_powerpoint)]

df_altman_powerpoint = df_altman_powerpoint.dropna().groupby(['id_s', 'measure'], as_index=False).median(numeric_only=True)

#df_altman_powerpoint['id_s_name'] = df_altman_powerpoint['id_s'].apply(lambda x: get_setting_axis_name(x))

df_altman_powerpoint.insert(0, 'id_s_name', df_altman_powerpoint['id_s'].apply(lambda x: get_setting_axis_name(x)))
df_altman_powerpoint.drop(columns=['id_s', 'mean_err'], inplace=True)
df_altman_powerpoint = df_altman_powerpoint[df_altman_powerpoint['measure'] != 'trunkDisplacementDEG']

df_altman_powerpoint['CAD'] = df_altman_powerpoint['measure'].apply(lambda x: df_cad[get_measure_short_name(x)].values[0])
df_altman_powerpoint['measure'] = df_altman_powerpoint['measure'].apply(lambda x: get_title_measure_name(x))
df_altman_powerpoint['LoA < CAD'] = df_altman_powerpoint.apply(
    lambda row: 'Yes' if (row['lower_loa'] > -row['CAD']) and (row['upper_loa'] < row['CAD']) else 'No', axis=1)

#round loa to 2 decimal places
df_altman_powerpoint['lower_loa'] = df_altman_powerpoint['lower_loa'].apply(lambda x: round(x, 2))
df_altman_powerpoint['upper_loa'] = df_altman_powerpoint['upper_loa'].apply(lambda x: round(x, 2))

df_altman_powerpoint.rename(columns={'id_s_name': 'Setting', 'measure': 'Measure', 'lower_loa': 'LoA Lower Bound', 'upper_loa': 'LoA Upper Bound'}, inplace=True)

df_altman_powerpoint.to_csv(os.path.join(dir_out, 'bland_altman_powerpoint.csv'),sep=';', index=False)

KeyError: "['mean_err'] not found in axis"

In [40]:
df_altman_powerpoint.columns

Index(['Setting', 'Measure', 'LoA Lower Bound', 'LoA Upper Bound', 'CAD',
       'LoA < CAD'],
      dtype='object')

In [41]:
print(df_altman_powerpoint.style.to_latex())

\begin{tabular}{lllrrrl}
 & Setting & Measure & LoA Lower Bound & LoA Upper Bound & CAD & LoA < CAD \\
0 & Metrabs, Cams: 1,2,3,4,5 & Maximum Elbow Extension Reaching & -16.030000 & 0.600000 & 7.820000 & No \\
1 & Metrabs, Cams: 1,2,3,4,5 & Interjoint Coordination & -0.240000 & 0.040000 & 0.180000 & No \\
2 & Metrabs, Cams: 1,2,3,4,5 & Number of Movement Units & -4.480000 & 7.060000 & 2.640000 & No \\
3 & Metrabs, Cams: 1,2,3,4,5 & Peak Endeffector Velocity & -306.190000 & 417.580000 & 78.960000 & No \\
4 & Metrabs, Cams: 1,2,3,4,5 & Shoulder Flexion Reaching & -10.680000 & -0.600000 & 4.480000 & No \\
5 & Metrabs, Cams: 1,2,3,4,5 & Peak Elbow Velocity & 0.300000 & 1.870000 & 21.710000 & Yes \\
6 & Metrabs, Cams: 1,2,3,4,5 & Maximum Shoulder Abduction Reaching & -6.130000 & 3.370000 & 9.990000 & Yes \\
7 & Metrabs, Cams: 1,2,3,4,5 & Shoulder Flexion Drinking & -15.690000 & -3.580000 & 6.850000 & No \\
8 & Metrabs, Cams: 1,2,3,4,5 & Relative time to First Peak Velocity & -87.490000 & -1

In [100]:
df_altman_powerpoint.pivot(index='id_s_name', columns='measure', values=['lower_loa', 'upper_loa'])

lower_loa                          \
measure                  Elbow Extension Interjoint Coordination   
id_s_name                                                          
Metrabs, Cams: 1,2,3,4,5      -16.029297               -0.239665   
Single, Cam: 1, filt          -19.071295               -0.308841   

                                                                       \
measure                  Number of Movement Units Peak Elbow Velocity   
id_s_name                                                               
Metrabs, Cams: 1,2,3,4,5                -4.717563            0.097556   
Single, Cam: 1, filt                    -3.470968            0.352483   

                                                    \
measure                  Peak Endeffector Velocity   
id_s_name                                            
Metrabs, Cams: 1,2,3,4,5               -304.771124   
Single, Cam: 1, filt                   -385.944691   

                                                               \
measure                  Relative time to First Peak Velocity   
id_s_name                                                       
Metrabs, Cams: 1,2,3,4,5                           -89.344026   
Single, Cam: 1, filt                               -91.150508   

                                                                  \
measure                  Relative time to Peak Velocity relative   
id_s_name                                                          
Metrabs, Cams: 1,2,3,4,5                              -45.684010   
Single, Cam: 1, filt                                  -41.782011   

                                                                       \
measure                  Shoulder Abduction Shoulder Flexion Reaching   
id_s_name                                                               
Metrabs, Cams: 1,2,3,4,5          -6.121695                -10.653790   
Single, Cam: 1, filt             -11.817173                -13.420837   

                                                      ...           upper_loa  \
measure                  Time to First Peak Velocity  ... Peak Elbow Velocity   
id_s_name                                             ...                       
Metrabs, Cams: 1,2,3,4,5                   -7.115463  ...            1.874313   
Single, Cam: 1, filt                       -7.451534  ...            2.076414   

                                                    \
measure                  Peak Endeffector Velocity   
id_s_name                                            
Metrabs, Cams: 1,2,3,4,5                416.557594   
Single, Cam: 1, filt                    625.730126   

                                                               \
measure                  Relative time to First Peak Velocity   
id_s_name                                                       
Metrabs, Cams: 1,2,3,4,5                           -12.439054   
Single, Cam: 1, filt                               -19.262908   

                                                                  \
measure                  Relative time to Peak Velocity relative   
id_s_name                                                          
Metrabs, Cams: 1,2,3,4,5                               37.177030   
Single, Cam: 1, filt                                   33.721099   

                                                                       \
measure                  Shoulder Abduction Shoulder Flexion Reaching   
id_s_name                                                               
Metrabs, Cams: 1,2,3,4,5           3.400755                 -0.650603   
Single, Cam: 1, filt               6.864447                 -0.254984   

                                                                            \
measure                  Time to First Peak Velocity Time to Peak Velocity   
id_s_name                                                                    
Metrabs, Cams: 1,2,3,4,5                   -0.827605              3.085358  

In [30]:
df_cad

,Unnamed: 0,mov_time,peak_V,peak_V_elb,t_to_PV,t_first_PV,t_PV_rel,t_first_PV_rel,n_mov_units,interj_coord,trunk_disp,arm_flex_reach,elb_ext,arm_abd,arm_flex_drink
0,CAD,2.18,78.96,21.71,0.13,0.08,5.86,8.72,2.64,0.18,31.89,4.48,7.82,9.99,6.85


In [21]:
df_latex = df_altman.dropna().groupby(['id_s', 'measure']).mean(numeric_only=True)

In [22]:
df_latex

mean_err     lower_loa     upper_loa
id_s measure                                                          
S001 ElbowExtension              11.294520     -1.896861     24.485901
     InterjointCoordination       0.082984     -0.221662      0.387630
     NumberMovementUnits          0.595074     -5.510456      6.700604
     PeakVelocity_mms          1856.080354   -852.635829   4564.796537
     ShoulderFlexionReaching    -10.812213    -20.520365     -1.104061
     elbowVelocity                1.503489     -1.136684      4.143661
     shoulderAbduction           16.816401      8.705198     24.927605
     shoulderFlexionDrinking    -10.390070    -23.920934      3.140794
     tToFirstpeakV_rel            0.135468    -39.441083     39.712018
     tToFirstpeakV_s             -0.003458     -3.000913      2.993997
     tTopeakV_rel                 0.135468    -39.441083     39.712018
     tTopeakV_s                  -0.003458     -3.000913      2.993997
     trunkDisplacementDEG         2.707858    -10.881653     16.297370
     trunkDisplacementMM      17150.596933 -48382.753209  82683.947074
S002 ElbowExtension              -6.536493    -16.085111      3.012126
     InterjointCoordination      -0.049280     -0.206787      0.108228
     NumberMovementUnits          1.422081     -5.151915      7.996077
     PeakVelocity_mms          2433.374542 -13668.645794  18535.394878
     ShoulderFlexionReaching     -6.521583    -16.551594      3.508428
     elbowVelocity                2.221827      0.424483      4.019171
     shoulderAbduction            0.828498     -7.756962      9.413958
     shoulderFlexionDrinking    -10.221559    -18.035859     -2.407259
     tToFirstpeakV_rel           -3.611113    -46.996230     39.774004
     tToFirstpeakV_s             -0.280434     -3.550087      2.989219
     tTopeakV_rel                -3.611113    -46.996230     39.774004
     tTopeakV_s                  -0.280434     -3.550087      2.989219
     trunkDisplacementDEG         1.778557    -10.167929     13.725042
     trunkDisplacementMM       9747.066378 -28835.816288  48329.949045
S017 ElbowExtension              -4.090169    -15.479185      7.298848
     InterjointCoordination      -0.091968     -0.317950      0.134013
     NumberMovementUnits          3.388028     -3.115393      9.891450
     PeakVelocity_mms           268.344867  -1957.117885   2493.807619
     ShoulderFlexionReaching     -7.595738    -15.417128      0.225652
     elbowVelocity                1.296444      0.473294      2.119594
     shoulderAbduction           -1.278776    -10.555185      7.997633
     shoulderFlexionDrinking    -11.766144    -19.269710     -4.262577
     tToFirstpeakV_rel           -2.959323    -44.628777     38.710131
     tToFirstpeakV_s             -0.233741     -3.454264      2.986783
     tTopeakV_rel                -2.959323    -44.628777     38.710131
     tTopeakV_s                  -0.233741     -3.454264      2.986783
     trunkDisplacementDEG        -0.484412     -7.669257      6.700434
     trunkDisplacementMM      18043.442444 -32188.076136  68274.961024
S018 ElbowExtension              -4.079164    -15.371307      7.212980
     InterjointCoordination      -0.091882     -0.317601      0.133836
     NumberMovementUnits          3.331475     -5.480783     12.143733
     PeakVelocity_mms          1514.312374  -5879.226303   8907.851051
     ShoulderFlexionReaching     -7.624342    -15.152993     -0.095692
     elbowVelocity                1.355281      0.364087      2.346475
     shoulderAbduction           -1.203746    -10.555646      8.148154
     shoulderFlexionDrinking    -11.717488    -19.267686     -4.167290
     tToFirstpeakV_rel          -11.099538    -61.700375     39.501298
     tToFirstpeakV_s             -0.875370     -4.861281      3.110541
     tTopeakV_rel               -11.099538    -61.700375     39.501298
     tTopeakV_s                  -0.875370     -4.861281      3.110541
     trunkDisplacementDEG        -0.483680     -7.655872  

In [23]:
print(df_latex.style.to_latex())

\begin{tabular}{llrrr}
 &  & mean_err & lower_loa & upper_loa \\
id_s & measure &  &  &  \\
\multirow[c]{14}{*}{S001} & ElbowExtension & 11.294520 & -1.896861 & 24.485901 \\
 & InterjointCoordination & 0.082984 & -0.221662 & 0.387630 \\
 & NumberMovementUnits & 0.595074 & -5.510456 & 6.700604 \\
 & PeakVelocity_mms & 1856.080354 & -852.635829 & 4564.796537 \\
 & ShoulderFlexionReaching & -10.812213 & -20.520365 & -1.104061 \\
 & elbowVelocity & 1.503489 & -1.136684 & 4.143661 \\
 & shoulderAbduction & 16.816401 & 8.705198 & 24.927605 \\
 & shoulderFlexionDrinking & -10.390070 & -23.920934 & 3.140794 \\
 & tToFirstpeakV_rel & 0.135468 & -39.441083 & 39.712018 \\
 & tToFirstpeakV_s & -0.003458 & -3.000913 & 2.993997 \\
 & tTopeakV_rel & 0.135468 & -39.441083 & 39.712018 \\
 & tTopeakV_s & -0.003458 & -3.000913 & 2.993997 \\
 & trunkDisplacementDEG & 2.707858 & -10.881653 & 16.297370 \\
 & trunkDisplacementMM & 17150.596933 & -48382.753209 & 82683.947074 \\
\multirow[c]{14}{*}{S002} & Elb

In [24]:
print(df_latex.to_markdown())

|                                     |       mean_err |     lower_loa |     upper_loa |
|:------------------------------------|---------------:|--------------:|--------------:|
| ('S001', 'ElbowExtension')          |    11.2945     |     -1.89686  |    24.4859    |
| ('S001', 'InterjointCoordination')  |     0.0829839  |     -0.221662 |     0.38763   |
| ('S001', 'NumberMovementUnits')     |     0.595074   |     -5.51046  |     6.7006    |
| ('S001', 'PeakVelocity_mms')        |  1856.08       |   -852.636    |  4564.8       |
| ('S001', 'ShoulderFlexionReaching') |   -10.8122     |    -20.5204   |    -1.10406   |
| ('S001', 'elbowVelocity')           |     1.50349    |     -1.13668  |     4.14366   |
| ('S001', 'shoulderAbduction')       |    16.8164     |      8.7052   |    24.9276    |
| ('S001', 'shoulderFlexionDrinking') |   -10.3901     |    -23.9209   |     3.14079   |
| ('S001', 'tToFirstpeakV_rel')       |     0.135468   |    -39.4411   |    39.712     |
| ('S001', 'tToFirstp